In [1]:
import pandas as pd
import numpy as np

In [2]:
ind_games = pd.read_csv("data/final_data/all_games_16_18.csv")

In [3]:
ind_games = ind_games.drop('Team2_int_re_number',axis=1)

In [4]:
clean_ind_games = ind_games.drop(['game_id','Team1_team_id','Team2_team_id','Team1','Team2'],axis=1)

### Target split

In [5]:
clean_ind_games['Team2_win'].value_counts()/clean_ind_games.shape[0]

1    0.622561
0    0.377439
Name: Team2_win, dtype: float64

### Re-shuffle dataframe to get equal target variables 

In [6]:
target_yes = clean_ind_games[clean_ind_games['Team2_win'] == 1]
target_no = clean_ind_games[clean_ind_games['Team2_win']==0]

In [7]:
sample_no = int((target_yes.shape[0] - target_no.shape[0])/2)

In [8]:
flip_df = target_yes.sample(n=sample_no)
remove_index = flip_df.index.tolist()

In [9]:
flip_df.head()

,Team1_team_score,Team2_team_score,Team1_first_downs,Team1_passing_first_downs,Team1_rushing_first_downs,Team1_rushing_yds,Team1_rushing_attempts,Team1_passing_attempts,Team1_passing_completions,Team1_passing_interceptions,...,Team2_third_down_suc,Team2_fourth_down_attempts,Team2_fourth_down_suc,Team1_third_down_conv_pct,Team2_third_down_conv_pct,Team1_fourth_down_conv_pct,Team2_fourth_down_conv_pct,Team1_turnover_margin,Team2_turnover_margin,Team2_win
143,17,33,18,6,9,83,34,35,15,2,...,3,1,1,0.375000,0.187500,0.333333,1.0,2,-2,1
621,13,16,23,5,17,63,27,39,27,1,...,6,2,1,0.333333,0.375000,0.000000,0.5,0,0,1
1804,17,24,15,4,8,85,34,28,18,1,...,7,2,2,0.500000,0.466667,1.000000,1.0,1,-1,1
712,16,32,11,6,3,139,45,19,7,3,...,6,1,1,0.333333,0.428571,0.500000,1.0,1,-1,1
1350,20,48,17,11,6,139,31,28,15,1,...,8,2,2,0.300000,0.500000,0.000000,1.0,1,-1,1


In [10]:
orig_cols = flip_df.columns.tolist()

In [11]:
target_yes = target_yes.drop(remove_index)

In [12]:
### Rename and flip values
t1_cols_rename = flip_df[flip_df.columns[flip_df.columns.str.contains('Team1')]].columns.tolist()
sep_df = flip_df[t1_cols_rename]
t1_cols_rename = [word.replace('1','2') for word in t1_cols_rename]
sep_df.columns = t1_cols_rename

t2_cols_rename = flip_df[flip_df.columns[flip_df.columns.str.contains('Team2')]].columns.tolist()
sep_df2 = flip_df[t2_cols_rename]
t2_cols_rename = [word.replace('2','1') for word in t2_cols_rename]
sep_df2.columns = t2_cols_rename

In [13]:
flip_df = pd.concat([sep_df,sep_df2],axis=1)
flip_df = flip_df.drop('Team1_win',axis=1)
flip_df['Team2_win'] = np.where(flip_df['Team2_team_score']>flip_df['Team1_team_score'],1,0)
flip_df = flip_df[orig_cols]

In [14]:
target_yes.shape

(1179, 63)

In [15]:
target_no = pd.concat([target_no,flip_df],axis=0)

In [16]:
clean_ind_games2 = pd.concat([target_yes,target_no],axis=0).sample(frac=1).reset_index(drop=True)

In [17]:
clean_ind_games2.to_csv('data/final_data/clean_shuffled_games.csv')

### Create Scale Model 

In [37]:
scale_df = clean_ind_games2.drop('Team2_win',axis=1).convert_objects(convert_numeric=True)
scale_df = scale_df.fillna(0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [38]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(scale_df)

In [42]:
from sklearn.externals import joblib 
scaler_filename = "models/scaler.sav"
joblib.dump(scaler, scaler_filename) 

['models/scaler.sav']

### Train/Test

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
trainingSet, testSet = train_test_split(clean_ind_games2, test_size=0.3,stratify=None)

In [20]:
trainingSet.to_csv("data/final_data/clean_train.csv")
testSet.to_csv("data/final_data/clean_test.csv")